In [1]:
import pandas as pd
import os
import pathlib

currentPath = pathlib.Path().resolve()
print(currentPath)

/workspaces/GolfStats


In [2]:
dir_path = os.path.abspath(os.path.join(currentPath,"data"))

fileName = "Edison's Scores.xlsx"

fileLocation = os.path.join(dir_path, fileName)

df = pd.read_excel(fileLocation, sheet_name='USKids')
print(df.head(3))

df['Date'] = pd.to_datetime(df['Date'])

dateDictionary = {}

divisions = set(df['Division'])
for division in divisions:
    subdf = df[df['Division'] == division]
    maxDate = subdf['Date'].max()
    minDate = subdf['Date'].min()
    dateDictionary[division] = {"Max":maxDate, "Min":minDate}

         Course Division       Date       Hole 1       Hole 2       Hole 3  \
0  Turtle Creek       U6 2022-04-10   3,70,2,3,5  4,115,5,4,9  5,190,4,4,8   
1     Sandridge       U6 2022-04-26  4,130,4,2,6  5,205,3,1,4   3,70,1,3,4   
2         Duran       U6 2022-05-01  5,200,4,4,8  4,121,3,3,6  4,116,2,3,5   

        Hole 4        Hole 5       Hole 6       Hole 7       Hole 8  \
0   3,77,2,4,6   5,200,5,2,7   3,65,1,3,4  4,110,2,3,5  5,205,4,3,7   
1  5,210,3,1,4  4,150,5,5,10   3,75,1,5,6  4,130,3,1,4  4,125,3,3,6   
2   3,65,1,2,3   4,110,2,3,5  4,120,2,3,5   3,77,4,5,9  4,109,2,2,4   

        Hole 9  Total  
0  4,120,4,2,6     57  
1  4,115,4,3,7     51  
2  5,205,4,1,5     50  


In [3]:
from typing import Optional

class HoleResult():
    def __init__(self, hole: str, input: str) -> None:
        holeResults : list[str] = input.split(",")
        self.hole : str = hole
        
        par = holeResults[0]
        if par[0] == "'":
            par = par.replace(par[0], "", 1)
            
        self.par : float = float(par)
        self.distance: float = float(holeResults[1])
        self.toGreen: float = float(holeResults[2])
        self.putts: float = float(holeResults[3])
        self.total: float = float(holeResults[4])
        
        self.fir : Optional[float] = None
        if len(holeResults) > 5:
            fir = holeResults[5]
            
            if fir == "X":
                self.fir = 0.0
            elif fir == "Y":
                self.fir = 1.0
            else:
                self.fir = None
        
class RoundResult():
    def __init__(self, location: str, division: str, date) -> None:
        self.location : str = location
        self.division : str = division
        self.date = date
        self.holes : dict[HoleResult] = {}
        
    def addHole(self, holeResult : HoleResult):
        self.holes[holeResult.hole] = holeResult

In [4]:
holes = ["Hole 1",	"Hole 2",	"Hole 3",	 "Hole 4",	"Hole 5",	"Hole 6",	"Hole 7",	"Hole 8",	"Hole 9"]


In [5]:
rounds:list[RoundResult] = []

#df = df.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    round = RoundResult(location = row['Course'], division=row['Division'], date=row['Date'])
    
        
    for holeName in holes:
        try:
            holeResult = HoleResult(hole=holeName, input=row[holeName])
            round.addHole(holeResult=holeResult)
        except:
            print("problem")
    rounds.append(round)

In [6]:
# Average Putts per Three Holes
distance = []
overUnder = []
score = []

division = []

dates_threehole = []
dates = []
datesFIR = []
puttAverage = []
scramblePerList = []
totalScore = []

avgPar3List = []
avgPar4List = []
avgPar5List = []

girList = []
firList = []


for round in rounds:
    my_dict : dict[HoleResult] = round.holes
    
    # Average per par type
    fir = 0
    gir = 0
    scrambleChance = 0
    scrambleSuccess = 0
    
    numberPar3 = 0
    totalPar3 = 0
    numberPar4 = 0
    totalPar4 = 0
    numberPar5 = 0
    totalPar5 = 0
    
    for hole in holes:
        currentHole = my_dict[hole]
        division.append(round.division)
        distance.append(currentHole.distance)
        overUnder.append(currentHole.total - currentHole.par)
        
        if currentHole.fir is not None:
            fir += currentHole.fir
        else:
            fir = None
        
        if currentHole.par == 3:
            numberPar3+=1
            if currentHole.toGreen == 1:
                gir+=1
            else:
                scrambleChance+=1
                
                if currentHole.putts == 1:
                    scrambleSuccess+=1
                
            totalPar3 += currentHole.total
            
        elif currentHole.par == 4:
            numberPar4 += 1
            if currentHole.toGreen == 2:
                gir+=1
            else:
                scrambleChance+=1
                
                if currentHole.putts == 1:
                    scrambleSuccess+=1
            
            totalPar4 += currentHole.total
            
        else:
            numberPar5 += 1
            if currentHole.toGreen == 3:
                gir+=1
            else:
                scrambleChance+=1
                
                if currentHole.putts == 1:
                    scrambleSuccess+=1
                
            totalPar5 += currentHole.total
    
    gir = gir/9
    girList.append(gir)

    if fir is not None:
        datesFIR.append(round.date)
        fir = fir/9
        firList.append(fir)
    
    scramblePer = scrambleSuccess/scrambleChance
    scramblePerList.append(scramblePer)
    
    avgPar3 = totalPar3/numberPar3
    avgPar4 = totalPar4/numberPar4
    avgPar5 = totalPar5/numberPar5
    
    avgPar3List.append(avgPar3)
    avgPar4List.append(avgPar4)
    avgPar5List.append(avgPar5)
    
    ## ============================================
    score = 0
    
    numberOfPutts = 0
    for hole in holes[0:3]:
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)
    
    numberOfPutts = 0
    for hole in holes[3:6]:
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)
    
    numberOfPutts = 0
    for hole in holes[6:9]:   
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    totalScore.append(score)
    dates.append(round.date)
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)


dfScoreOverDistance = pd.DataFrame(list(zip(distance, overUnder, division)),
                                   columns=['Distance', 'OverUnder', 'Divison'])

dfScrambleAvg =  pd.DataFrame(list(zip(dates, scramblePerList)),
               columns =['Date', 'Scramble'])  
dfScrambleAvg.set_index('Date', inplace= True)
    
dfGirAverage =  pd.DataFrame(list(zip(dates, girList)),
               columns =['Date', 'GIR'])   
dfGirAverage.set_index('Date', inplace= True)


dfFirAverage =  pd.DataFrame(list(zip(datesFIR,firList)),
               columns =['Date', 'FIR'])   
dfFirAverage.set_index('Date', inplace= True)


dfHoleAverage = pd.DataFrame(list(zip(dates, avgPar3List, avgPar4List, avgPar5List)),
               columns =['Date', 'Par 3', 'Par 4', 'Par 5'])    
dfHoleAverage.set_index('Date', inplace= True)

dfPutts = pd.DataFrame(list(zip(dates_threehole, puttAverage)),
               columns =['Date', 'Putts'])
dfPutts.set_index('Date', inplace= True)

dfScore = pd.DataFrame(list(zip(dates, totalScore)),
               columns =['Date', 'Score'])

dfScore.set_index('Date', inplace= True)

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [ ]:
colors = "bgrcmykw"

def get_cmap(n, name = 'hsv'):
    return plt.cm.get_cmap(name,n)

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Distance [yd]")
plt.ylabel("OverUnder")
plt.grid(True)

for i, (division) in enumerate(divisions):
    mask = dfScoreOverDistance["Divison"] == division   
    localDivision = dfScoreOverDistance[mask]
    ax.scatter(localDivision['Distance'], localDivision['OverUnder'], label=division)

ax.legend()

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Mean Putts over Three Holes")
plt.grid(True)
dateDictionary[division] = {"Max":maxDate, "Min":minDate}

for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
    
ax.scatter(dfPutts.index, dfPutts['Putts'])

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Mean Score")
plt.grid(True)
for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
   
   
ax.scatter(dfHoleAverage.index, dfHoleAverage['Par 3'], label='Par 3')
ax.scatter(dfHoleAverage.index, dfHoleAverage['Par 4'], label='Par 4')
ax.scatter(dfHoleAverage.index, dfHoleAverage['Par 5'], label='Par 5')
ax.legend()

print(dfHoleAverage)

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Total Score")
plt.grid(True)
for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
   
   
ax.scatter(dfScore.index, dfScore['Score'])

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("GIR")
plt.grid(True)
for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
   
   
ax.scatter(dfGirAverage.index, dfGirAverage['GIR'])

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("FIR")
plt.grid(True)
for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
   
   
ax.scatter(dfFirAverage.index, dfFirAverage['FIR'])

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Scramble")
plt.grid(True)
for i, (division) in enumerate(divisions):
    localDict = dateDictionary[division]
    plt.axvspan(localDict["Min"], localDict["Max"], color = colors[i], alpha = 0.2)
   
   
ax.scatter(dfScrambleAvg.index, dfScrambleAvg['Scramble'])